In [ ]:
# Jupyter Notebook

# Importando as bibliotecas necessárias
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

# Configurações iniciais
sns.set(style="whitegrid")
pd.options.mode.chained_assignment = None  # Para evitar warnings de cópias de DataFrame

# Configurando os diretórios
input_dir = Path("../runs")
output_dir = Path("../results")
output_dir.mkdir(exist_ok=True)

# Obtendo os arquivos CSV
csv_files = list(input_dir.glob("*.csv"))

# Gerando o DataFrame consolidado
runs_df = pd.concat([pd.read_csv(file) for file in csv_files], ignore_index=True)
runs_df_or = runs_df.copy()
runs_df = runs_df.dropna()

# Calculando a diferença de payoffs
runs_df["diff_payoffs"] = runs_df["my_payoff"] - runs_df["other_payoff"]

# Agrupando os dados para calcular médias e somas de payoffs
sum_runs = (
    runs_df.groupby(["scenario", "step", "strategy_name"])
    .agg(
        payoff_mean=("my_payoff", "mean"),
        payoff_sum=("my_payoff", "sum"),
    )
    .reset_index()
)

# Calculando cooperação e defecção
sum_cooperation = (
    runs_df.groupby(["scenario", "step"])
    .agg(
        my_play_C=("my_play", lambda x: (x == "C").sum()),
        my_play_D=("my_play", lambda x: (x == "D").sum()),
        ot_play_C=("other_play", lambda x: (x == "C").sum()),
        ot_play_D=("other_play", lambda x: (x == "D").sum()),
        players=("my_play", "count"),
    )
    .reset_index()
)

sum_cooperation["cooperation"] = sum_cooperation["my_play_C"] / sum_cooperation["players"]
sum_cooperation["defection"] = sum_cooperation["my_play_D"] / sum_cooperation["players"]

# Gráfico 1: Payoff médio por estratégia ao longo do tempo
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sum_runs, x="step", y="payoff_mean", hue="strategy_name", alpha=0.1)
sns.lineplot(data=sum_runs, x="step", y="payoff_mean", hue="strategy_name", legend=False)
plt.title("Payoff Médio por Estratégia ao Longo do Tempo")
plt.xlabel("Passo")
plt.ylabel("Payoff Médio")
plt.legend(title="Estratégia")
plt.savefig(output_dir / "payoff_mean_by_strategy.png")
plt.show()

# Gráfico 2: Distribuição de payoff médio por estratégia
plt.figure(figsize=(10, 6))
sns.violinplot(data=sum_runs, x="strategy_name", y="payoff_mean", inner=None)
sns.boxplot(data=sum_runs, x="strategy_name", y="payoff_mean", width=0.2, color="white")
sns.pointplot(
    data=sum_runs,
    x="strategy_name",
    y="payoff_mean",
    color="red",
    join=False,
    markers="o",
    scale=1.5,
)
plt.title("Distribuição de Payoff Médio por Estratégia")
plt.xlabel("Estratégia")
plt.ylabel("Payoff Médio")
plt.savefig(output_dir / "payoff_distribution_by_strategy.png")
plt.show()

# Gráfico 3: Cooperação ao longo do tempo
plt.figure(figsize=(10, 6))
sns.scatterplot(data=sum_cooperation, x="step", y="cooperation", alpha=0.5)
sns.lineplot(data=sum_cooperation, x="step", y="cooperation", color="blue")
plt.title("Cooperação ao Longo do Tempo")
plt.xlabel("Passo")
plt.ylabel("Cooperação")
plt.savefig(output_dir / "cooperation_over_time.png")
plt.show()